### Google Cloud Vision API 제품검색 기능

 -----------------

#### ~ Google Vision API 연결 되었는지 체크

In [5]:
from google.cloud import vision
import argparse
GOOGLE_APPLICATION_CREDENTIALS= 'C/Users/ghkdn/Jupyter/ESTSOFT/Google_Vision_AI/last-275112-e392011724ba.json'

In [6]:
def run_quickstart(file_name):
    import io
    import os
 
    # 구글 라이브러리 import
    from google.cloud import vision
    from google.cloud.vision import types
 
    # 사용할 클라이언트 설정
    client = vision.ImageAnnotatorClient()
    
    # 이미지 읽기
    with io.open(file_name, 'rb') as image_file:
        content = image_file.read()
 
    image = types.Image(content=content)
 
    # label 뽑아냄.
    response = client.label_detection(image=image)
    labels = response.label_annotations
 
    print('Labels:')
    for label in labels:
        print(label.description + " = " + str(int(label.score*100)) + "%")
 
if __name__ == '__main__':
    run_quickstart("the_bestJazz-Clubs_in_newYorkCit.jpg")


Labels:
Red = 91%
Illustration = 89%
Graphic design = 88%
Art = 86%
Font = 74%
Design = 73%
Graffiti = 71%
Fictional character = 69%
Graphics = 69%
Visual arts = 68%


#### ~Google Storage 탐색 

In [7]:
# storage 탐색
# Blob : 저장할 파일 이름입니다.
from google.cloud import storage
storage_client = storage.Client()
bucket = storage_client.bucket('hwao-bucket')
blobs = bucket.list_blobs()
for blob in blobs:
    print(blob.name)

BEACH-B-S(48)_01.jpg
BEACH-B-S(48)_02.jpg
BEACH-B-S(48)_03.jpg
BEACH-B-S(48)_04.jpg
BOSS1067_F-124_01.jpg
BOSS1067_F-124_02.jpg
BOSS1067_F-124_03.jpg
BOSS1067_F-124_04.jpg
D1226-C01/D1226-C01_01.jpg
D1226-C01/D1226-C01_02.jpg
D1226-C01/D1226-C01_03.jpg
D1226-C01/D1226-C01_04.jpg
MB618F-001_01.jpg
MB618F-001_02.jpg
MB618F-001_03.jpg
MB618F-001_04.jpg
RB3447V-2620(50)_01.jpg
RB3447V-2620(50)_02 (1).jpg
RB3447V-2620(50)_02.jpg
RB3447V-2620(50)_03.jpg
RB3447V-2620(50)_04.jpg
RB5154-2000(51)_01.jpg
RB5154-2000(51)_02 (1).jpg
RB5154-2000(51)_02.jpg
RB5154-2000(51)_03.jpg
RB5154-2000(51)_04.jpg
RB6369D-2509_01 (1).jpg
RB6369D-2509_01.jpg
RB6369D-2509_02.jpg
RB6369D-2509_03 (1).jpg
RB6369D-2509_03.jpg
RB6369D-2509_04 (1).jpg
RB6369D-2509_04.jpg
VPR54T-1BO-1O1_01.jpg
VPR54T-1BO-1O1_02.jpg
VPR54T-1BO-1O1_03.jpg
VPR54T-1BO-1O1_04.jpg
gsc_csv.csv
gsc_csv_v2.csv
gsc_csv_v3.csv
gsc_csv_v4.csv


##### 1. CSV 데이터 파일 일괄적으로 불러오기  >  PRODUCT SET 및 PRODUCT 동시 생성

In [4]:
def import_product_sets(project_id, location, gcs_uri):
        """Import images of different products in the product set.
        Args:
            project_id: Id of the project.
            location: A compute region name.
            gcs_uri: Google Cloud Storage URI.
                Target files must be in Product Search CSV format.
        """
        client = vision.ProductSearchClient()

        # A resource that represents Google Cloud Platform location.
        location_path = client.location_path(
            project=project_id, location=location)

        # Set the input configuration along with Google Cloud Storage URI
        gcs_source = vision.types.ImportProductSetsGcsSource(
            csv_file_uri=gcs_uri)
        input_config = vision.types.ImportProductSetsInputConfig(
            gcs_source=gcs_source)

        # Import the product sets from the input URI.
        response = client.import_product_sets(
            parent=location_path, input_config=input_config)

        print('Processing operation name: {}'.format(response.operation.name))
        # synchronous check of operation status
        result = response.result()
        print('Processing done.')

        for i, status in enumerate(result.statuses):
            print('Status of processing line {} of the csv: {}'.format(
                i, status))
            # Check the status of reference image
            # `0` is the code for OK in google.rpc.Code.
            if status.code == 0:
                reference_image = result.reference_images[i]
                print(reference_image)
            else:
                print('Status code not OK: {}'.format(status.message))

In [5]:
import_product_sets('last-275112','us-east1','gs://hwao-bucket/item_data.csv')

Processing operation name: projects/last-275112/locations/us-east1/operations/84ce70ab1d29dc20
Processing done.
Status of processing line 0 of the csv: 
name: "projects/last-275112/locations/us-east1/products/BEACH-B-S(48)/referenceImages/item0"
uri: "gs://hwao-bucket/BEACH-B-S(48)_01.jpg"
bounding_polys {
}

Status of processing line 1 of the csv: 
name: "projects/last-275112/locations/us-east1/products/BEACH-B-S(48)/referenceImages/item1"
uri: "gs://hwao-bucket/BEACH-B-S(48)_02.jpg"
bounding_polys {
}

Status of processing line 2 of the csv: 
name: "projects/last-275112/locations/us-east1/products/BEACH-B-S(48)/referenceImages/item2"
uri: "gs://hwao-bucket/BEACH-B-S(48)_03.jpg"
bounding_polys {
}

Status of processing line 3 of the csv: 
name: "projects/last-275112/locations/us-east1/products/BEACH-B-S(48)/referenceImages/item3"
uri: "gs://hwao-bucket/BEACH-B-S(48)_04.jpg"
bounding_polys {
}

Status of processing line 4 of the csv: 
name: "projects/last-275112/locations/us-east1/prod

#### ~ 프로젝트의 모든 제품 나열

In [4]:
def list_product_sets(project_id, location):
    """List all product sets.
    Args:
        project_id: Id of the project.
        location: A compute region name.
    """
    client = vision.ProductSearchClient()

    # A resource that represents Google Cloud Platform location.
    location_path = client.location_path(
        project=project_id, location=location)

    # List all the product sets available in the region.
    product_sets = client.list_product_sets(parent=location_path)

    # Display the product set information.
    for product_set in product_sets:
        print('Product set name: {}'.format(product_set.name))
        print('Product set id: {}'.format(product_set.name.split('/')[-1]))
        print('Product set display name: {}'.format(product_set.display_name))
        print('Product set index time:')
        print('  seconds: {}'.format(product_set.index_time.seconds))
        print('  nanos: {}\n'.format(product_set.index_time.nanos))
# [END vision_product_search_list_product_sets]

In [5]:
list_product_sets('last-275112','us-east1')

Product set name: projects/last-275112/locations/us-east1/productSets/PS_CLOTH-SHOE_070318
Product set id: PS_CLOTH-SHOE_070318
Product set display name: CLOTH-SHOE
Product set index time:
  seconds: 1587941682
  nanos: 762382807

Product set name: projects/last-275112/locations/us-east1/productSets/house_glass
Product set id: house_glass
Product set display name:  
Product set index time:
  seconds: 1587941682
  nanos: 762382807

Product set name: projects/last-275112/locations/us-east1/productSets/house_glass1
Product set id: house_glass1
Product set display name:  
Product set index time:
  seconds: 1587941682
  nanos: 762382807

Product set name: projects/last-275112/locations/us-east1/productSets/product_set0
Product set id: product_set0
Product set display name:  
Product set index time:
  seconds: 1587941682
  nanos: 762382807



#### 2. 유사 제품 이미지 가져오기

In [6]:
# [START vision_product_search_get_similar_products]
def get_similar_products_file(
        project_id, location, product_set_id, product_category,
        file_path, filter):

    """Search similar products to image.
    Args:
        project_id: Id of the project.
        location: A compute region name.
        product_set_id: Id of the product set.
        product_category: Category of the product.
        file_path: Local file path of the image to be searched.
        filter: Condition to be applied on the labels.
        Example for filter: (color = red OR color = blue) AND style = kids
        It will search on all products with the following labels:
        color:red AND style:kids
        color:blue AND style:kids
    """
    # product_search_client is needed only for its helper methods.
    product_search_client = vision.ProductSearchClient()
    image_annotator_client = vision.ImageAnnotatorClient()

    # Read the image as a stream of bytes.
    with open(file_path, 'rb') as image_file:
        content = image_file.read()

    # Create annotate image request along with product search feature.
    image = vision.types.Image(content=content)

    # product search specific parameters
    product_set_path = product_search_client.product_set_path(
        project=project_id, location=location,
        product_set=product_set_id)
    product_search_params = vision.types.ProductSearchParams(
        product_set=product_set_path,
        product_categories=[product_category],
        filter=filter)
    image_context = vision.types.ImageContext(
        product_search_params=product_search_params)

    # Search products similar to the image.
    response = image_annotator_client.product_search(
        image, image_context=image_context)

    index_time = response.product_search_results.index_time
    print('Product set index time:')
    print('  seconds: {}'.format(index_time.seconds))
    print('  nanos: {}\n'.format(index_time.nanos))

    results = response.product_search_results.results

    print('Search results:')
    
    score_dict = {}
    for result in results:
        product = result.product

        print('Score(Confidence): {}'.format(result.score))
        print('Image name: {}'.format(result.image))

        print('Product name: {}'.format(product.name))
        print('Product display name: {}'.format(
            product.display_name))
        print('Product description: {}\n'.format(product.description))
        print('Product labels: {}\n'.format(product.product_labels))
        
        product_name = product.name
        product_name = str(product_name)      
        score_dict[product_name] = result.score
    return score_dict
# [END vision_product_search_get_similar_products]


In [23]:
result_dict = get_similar_products_file('last-275112','us-east1','house_glass','general-v1','C:/Users/ghkdn/Jupyter/ESTSOFT/Google_Vision_AI/sample0_BOSS1067_F-124.jpg','category=glass')

Product set index time:
  seconds: 1587941682
  nanos: 762382807

Search results:
Score(Confidence): 0.40210726857185364
Image name: projects/last-275112/locations/us-east1/products/VPR54T-1BO-1O1/referenceImages/item32
Product name: projects/last-275112/locations/us-east1/products/VPR54T-1BO-1O1
Product display name: glass_display
Product description: 

Product labels: [key: "category"
value: "glass"
]

Score(Confidence): 0.3925269544124603
Image name: projects/last-275112/locations/us-east1/products/BOSS1067_F-124/referenceImages/item4
Product name: projects/last-275112/locations/us-east1/products/BOSS1067_F-124
Product display name: glass_display
Product description: 

Product labels: [key: "category"
value: "glass"
]

Score(Confidence): 0.36212295293807983
Image name: projects/last-275112/locations/us-east1/products/RB6369D-2509/referenceImages/item16
Product name: projects/last-275112/locations/us-east1/products/RB6369D-2509
Product display name: glass_display
Product description:

In [24]:
image_url = max(result_dict.keys(), key = (lambda k: result_dict[k]) )
print("가장 유사한 이미지 파일 경로: {}".format(image_url))

model_name = image_url.split("/")[-1] # 생성되는 image_url 마지막이 모델명으로 담김
print("가장 유사한 모델: {}".format(model_name))


가장 유사한 이미지 파일 경로: projects/last-275112/locations/us-east1/products/VPR54T-1BO-1O1
가장 유사한 모델: VPR54T-1BO-1O1


In [11]:
'''
제품의 제품 검색 색인은 약 30분마다 업데이트됩니다. 
이미지가 추가되거나 삭제되어도 다음 번에 색인이 업데이트될 때까지 제품검색 응답에 변경사항이 반영되지 않습니다.
'''

'\n제품의 제품 검색 색인은 약 30분마다 업데이트됩니다. \n이미지가 추가되거나 삭제되어도 다음 번에 색인이 업데이트될 때까지 제품검색 응답에 변경사항이 반영되지 않습니다.\n'